**Requirements:**

*   Login to your Hugging Face account
*   API access key
*   !pip install -q transformers accelerate bitsandbytes torch



In [ ]:
#Microsoft's Phi-3-mini-4k-instruct

import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

CODEBOOK = """Evaluation Criteria (0-4 scale):
1. Stylistic Similarity: Grammar, vocabulary, sentence structure, tone, punctuation, capitalization, emojis, hashtags
2. Topic Similarity: Main subject matter and theme alignment
3. Sentiment Similarity: Emotional tone, intensity and attitude

Rating Guide:
4 = Very Similar, 3 = Similar, 2 = Neutral, 1 = Dissimilar, 0 = Very Dissimilar
"""

def format_prompt(original, synthetic):
    return f"""<|user|>
{CODEBOOK}

Analyze this tweet pair. Provide ONLY three numerical scores (0-4) separated by commas in this exact format:
"stylistic,topic,sentiment"

Original: {original}
Synthetic: {synthetic}
<|assistant|>
Scores: """

def evaluate_pair(original, synthetic):
    prompt = format_prompt(original, synthetic)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=15,
        do_sample=False,
        temperature=0.01,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    scores = re.findall(r'\b[0-4]\b', response.split("Scores: ")[-1])

def batch_evaluate(tweet_pairs):
    results = []
    for pair in tweet_pairs:
        scores = evaluate_pair(pair["original"], pair["synthetic"])
        results.append({
            "original": pair["original"],
            "synthetic": pair["synthetic"],
            "style": scores[0],
            "topic": scores[1],
            "sentiment": scores[2]
        })
    return pd.DataFrame(results)

def read_tweet_pairs_from_excel(file_path):
    """
    Reads tweet pairs from an Excel file.

    Parameters:
    - file_path: Path to the Excel file.

    Returns:
    - A list of dictionaries containing the tweet pairs.
    """
    try:
        # Read the Excel file
        df = pd.read_excel(file_path)

        # Ensure the required columns exist
        required_columns = ["original", "synthetic"]
        if not all(col in df.columns for col in required_columns):
            raise ValueError("The Excel file must contain 'original' and 'synthetic' columns.")

        # Convert DataFrame to a list of dictionaries
        tweet_pairs = df.to_dict('records')

        return tweet_pairs

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

def main():
    file_path = "File_Name.xlsx"  # Path to your Excel file
    tweet_pairs = read_tweet_pairs_from_excel(file_path)

    if tweet_pairs:
        results_df = batch_evaluate(tweet_pairs)
        print(results_df)

        # Save results to a CSV file
        output_file_path = "Result.csv"
        results_df.to_csv(output_file_path, index=False)
        print(f"Results saved to {output_file_path}")

        if not results_df.empty:
            print("\nAverage Scores:")
            print(results_df[['style', 'topic', 'sentiment']].mean())
    else:
        print("No tweet pairs found.")

if __name__ == "__main__":
    main()

In [ ]:
#Llama-3.2-1B-Instruct

#pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "User_Prompt"},
]
pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B-Instruct")
pipe(messages)

In [ ]:
import os
import torch
from huggingface_hub import login

# Set your Hugging Face token as an environment variable
os.environ["HUGGINGFACE_TOKEN"] = "TOKEN_HERE"

# Or login using the huggingface_hub library
login(token="TOKEN_HERE")
model="meta-llama/Llama-3.2-1B-Instruct"
# Then load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", token=os.environ["HUGGINGFACE_TOKEN"])
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B-Instruct",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,
    attn_implementation="sdpa",
    token=os.environ["HUGGINGFACE_TOKEN"]
)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
import re

MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"
DEVICE = "cuda"

# Load model with 4-bit quantization for T4 compatibility
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,  # Optimize for T4's 16GB VRAM
    attn_implementation="sdpa"  #Flash attention
)

CODEBOOK = """Evaluation Criteria (0-4):
1. Stylistic: Writing style elements
2. Topic: Main subject alignment
3. Sentiment: Emotional tone match

Rating Guide:
4 = Very Similar, 3 = Similar, 2 = Neutral, 1 = Dissimilar, 0 = Very Dissimilar"""

"""
Customize the CODEBOOK accordingly:
CODEBOOK = Evaluation Criteria (0-4 scale):
1. Stylistic Similarity: Grammar, vocabulary, sentence structure, tone, punctuation, capitalization, emojis, hashtags
2. Topic Similarity: Main subject matter and theme alignment
3. Sentiment Similarity: Emotional tone, intensity and attitude

Rating Guide:
4 = Very Similar, 3 = Similar, 2 = Neutral, 1 = Dissimilar, 0 = Very Dissimilar
"""

def generate_prompt(original, synthetic):
    return f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert evaluator comparing tweets. {CODEBOOK}<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Original: {original}
Synthetic: {synthetic}
Provide scores as: stylistic,topic,sentiment<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
Scores: """

def extract_scores(response):
    # Find first 3 numbers between 0-4 in response
    numbers = re.findall(r'\b[0-4]\b', response)
    return [int(n) for n in numbers[:3]] if len(numbers)>=3 else [None]*3

def evaluate_pair(original, synthetic):
    prompt = generate_prompt(original, synthetic)

    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)

    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=30,
            temperature=0.1,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return extract_scores(response.split("Scores: ")[-1])

# Batch processing function
def evaluate_tweets(tweet_pairs):
    results = []
    for pair in tweet_pairs:
        style, topic, sentiment = evaluate_pair(
            pair["original"],
            pair["synthetic"]
        )
        results.append({
            "original": pair["original"],
            "synthetic": pair["synthetic"],
            "style_score": style,
            "topic_score": topic,
            "sentiment_score": sentiment
        })
    return pd.DataFrame(results)

def read_tweet_pairs_from_excel(file_path):
    """
    Reads tweet pairs from an Excel file.

    Parameters:
    - file_path: Path to the Excel file.

    Returns:
    - A list of dictionaries containing the tweet pairs.
    """
    try:
        # Read the Excel file
        df = pd.read_excel(file_path)

        # Ensure the required columns exist
        required_columns = ["original", "synthetic"]
        if not all(col in df.columns for col in required_columns):
            raise ValueError("The Excel file must contain 'original' and 'synthetic' columns.")

        # Convert DataFrame to a list of dictionaries
        tweet_pairs = df.to_dict('records')

        return tweet_pairs

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

def main():
    file_path = "File_Name.xlsx"  # Path to your Excel file
    tweet_pairs = read_tweet_pairs_from_excel(file_path)

    if tweet_pairs:
        results_df = evaluate_tweets(tweet_pairs)
        print("Evaluation Results:")
        print(results_df)

        # Save results to a CSV file
        output_file_path = "Results.csv"
        results_df.to_csv(output_file_path, index=False)
        print(f"Results saved to {output_file_path}")

        if not results_df.empty:
            print("\nAverage Scores:")
            print(results_df[['style_score', 'topic_score', 'sentiment_score']].mean())
    else:
        print("No tweet pairs found.")

if __name__ == "__main__":
    main()